In [23]:
# Install library yang dibutuhkan
# pip install nltk
# pip install Sastrawi
# pip install stanza

import string
import numpy as np
import pandas as pd
from sklearn.preprocessing import normalize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics.pairwise import cosine_similarity
from scipy.spatial.distance import euclidean
import re
import nltk
import stanza
from nltk.corpus import stopwords
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

# Download data nltk
nltk.download('stopwords')
nltk.download('omw-1.4')

# Inisialisasi stopwords, stemmer, and lemmatizer
stop_words = set(stopwords.words('indonesian'))
factory = StemmerFactory()
stemmer = factory.create_stemmer()
stanza.download('id')
nlp = stanza.Pipeline('id')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ACER\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\ACER\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
2024-10-13 11:05:06 INFO: Downloaded file to C:\Users\ACER\stanza_resources\resources.json
2024-10-13 11:05:06 INFO: Downloading default packages for language: id (Indonesian) ...
2024-10-13 11:05:08 INFO: File exists: C:\Users\ACER\stanza_resources\id\default.zip
2024-10-13 11:05:12 INFO: Finished downloading models and saved to C:\Users\ACER\stanza_resources
2024-10-13 11:05:12 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES
2024-10-13 11:05:12 INFO: Downloaded file to C:\Users\ACER\stanza_resources\resources.json
2024-10-13 11:05:1

UI Dengan Streamlit

In [27]:
# Install library yang dibutuhkan
# Buat file app.py di dalam jupyter notebook
# eksekusi streamlit dengan aktivasi enviroment tempat streamlit diinstall (.venv\Scripts\python -m streamlit run app_lsi.py)

%%writefile app_lsi.py
import streamlit as st
import string
import numpy as np
import pandas as pd
from sklearn.preprocessing import normalize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics.pairwise import cosine_similarity
from scipy.spatial.distance import euclidean
import re
import nltk
import stanza
from nltk.corpus import stopwords
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

# Download data nltk
nltk.download('stopwords')
nltk.download('omw-1.4')

# Inisialisasi stopwords, stemmer, and lemmatizer
stop_words = set(stopwords.words('indonesian'))
factory = StemmerFactory()
stemmer = factory.create_stemmer()
stanza.download('id')
nlp = stanza.Pipeline('id')

# Preprocessing functions
def case_folding(text):
    text = text.lower()
    return re.sub(r'[^\w\s]', '', text)

def tokenize(text):
    return text.split()

def remove_stopwords(tokens):
    return [word for word in tokens if word not in stop_words]

def stemming(tokens):
    return [stemmer.stem(word) for word in tokens]

def lemmatization(tokens):
    text = ' '.join(tokens)
    doc = nlp(text)
    return [word.lemma for sent in doc.sentences for word in sent.words]

def preprocess_with_stemming(text):
    text = case_folding(text)
    tokens = tokenize(text)
    tokens = remove_stopwords(tokens)
    tokens = stemming(tokens)
    return ' '.join(tokens)

def preprocess_with_lemmatization(text):
    text = case_folding(text)
    tokens = tokenize(text)
    tokens = remove_stopwords(tokens)
    tokens = lemmatization(tokens)
    return ' '.join(tokens)

# TF-IDF functions
def tfidf_with_stemming(documents):
    preprocessed_documents = [preprocess_with_stemming(doc) for doc in documents]
    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform(preprocessed_documents)
    return tfidf_matrix, vectorizer

def tfidf_with_lemmatization(documents):
    preprocessed_documents = [preprocess_with_lemmatization(doc) for doc in documents]
    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform(preprocessed_documents)
    return tfidf_matrix, vectorizer

# TF functions
def tf_with_lemmatization(documents):
    preprocessed_documents = [preprocess_with_lemmatization(doc) for doc in documents]
    vectorizer = CountVectorizer()
    tf_matrix = vectorizer.fit_transform(preprocessed_documents)
    return tf_matrix, vectorizer

def binary_tf_with_lemmatization(documents):
    preprocessed_documents = [preprocess_with_lemmatization(doc) for doc in documents]
    vectorizer = CountVectorizer(binary=True)
    binary_tf_matrix = vectorizer.fit_transform(preprocessed_documents)
    return binary_tf_matrix, vectorizer

def log_tf_with_lemmatization(documents):
    preprocessed_documents = [preprocess_with_lemmatization(doc) for doc in documents]
    vectorizer = CountVectorizer()
    tf_matrix = vectorizer.fit_transform(preprocessed_documents).toarray()
    log_tf_matrix = np.log1p(tf_matrix)
    return log_tf_matrix, vectorizer

def augmented_tf_with_lemmatization(documents):
    preprocessed_documents = [preprocess_with_lemmatization(doc) for doc in documents]
    vectorizer = CountVectorizer()
    tf_matrix = vectorizer.fit_transform(preprocessed_documents).toarray()
    augmented_tf_matrix = 0.5 + 0.5 * (tf_matrix / tf_matrix.max(axis=1, keepdims=True))
    return augmented_tf_matrix, vectorizer

# LSI search functions
def lsi_search(documents, query, vectorizer_func, n_components=20, similarity_metric='cosine'):
    matrix, vectorizer = vectorizer_func(documents)
    
    svd = TruncatedSVD(n_components=n_components, random_state=42)
    lsi_matrix = svd.fit_transform(matrix)
    
    preprocessed_query = preprocess_with_lemmatization(query)
    query_vec = vectorizer.transform([preprocessed_query])
    query_lsi = svd.transform(query_vec)
    
    if similarity_metric == 'cosine':
        similarities = cosine_similarity(query_lsi, lsi_matrix)[0]
        ranked_docs = np.argsort(similarities)[::-1]
        scores = similarities
    elif similarity_metric == 'euclidean':
        distances = [euclidean(query_lsi[0], doc_vec) for doc_vec in lsi_matrix]
        ranked_docs = np.argsort(distances)
        scores = distances
    
    return ranked_docs, scores

# Streamlit app
def main():
    st.title("LSI-based Document Search System")
    
    # Input for documents
    documents = st.text_area("Enter your documents (one per line):", height=200)
    documents = documents.split('\n')
    
    # Input for query
    query = st.text_input("Enter your search query:")
    
    # Select vectorization method
    vectorization_method = st.selectbox(
        "Select vectorization method:",
        ["TF-IDF (Lemmatization)", "TF (Lemmatization)", "Binary TF (Lemmatization)", "Log TF (Lemmatization)", "Augmented TF (Lemmatization)"]
    )
    
    # Select number of LSI components
    n_components = st.slider("Number of LSI components:", min_value=2, max_value=50, value=20)
    
    # Select similarity metric
    similarity_metric = st.radio("Select similarity metric:", ["Cosine Similarity", "Euclidean Distance"])
    
    if st.button("Search"):
        if documents and query:
            # Choose vectorization function
            if vectorization_method == "TF-IDF (Lemmatization)":
                vectorizer_func = tfidf_with_lemmatization
            elif vectorization_method == "TF (Lemmatization)":
                vectorizer_func = tf_with_lemmatization
            elif vectorization_method == "Binary TF (Lemmatization)":
                vectorizer_func = binary_tf_with_lemmatization
            elif vectorization_method == "Log TF (Lemmatization)":
                vectorizer_func = log_tf_with_lemmatization
            else:  # Augmented TF (Lemmatization)
                vectorizer_func = augmented_tf_with_lemmatization
            
            # Perform search
            ranked_docs, scores = lsi_search(
                documents, 
                query, 
                vectorizer_func, 
                n_components=n_components, 
                similarity_metric='cosine' if similarity_metric == "Cosine Similarity" else 'euclidean'
            )
            
            # Display results
            st.subheader("Search Results:")
            for i, idx in enumerate(ranked_docs[:10]):  # Display top 10 results
                score = scores[idx]
                if similarity_metric == "Cosine Similarity":
                    st.write(f"{i+1}. (Similarity: {score:.4f}) {documents[idx]}")
                else:
                    st.write(f"{i+1}. (Euclidean Distance: {score:.4f}) {documents[idx]}")
        else:
            st.warning("Please enter both documents and a search query.")
        
        # Preprocessing demonstration
    if st.checkbox("Show Preprocessing Results"):
        st.subheader("Preprocessing Results:")
        for doc in documents:
            original_text = case_folding(doc)
            tokens = tokenize(original_text)
            tokens_no_stopwords = remove_stopwords(tokens)
            stemmed_result = stemming(tokens_no_stopwords)
            lemmatized_result = lemmatization(tokens_no_stopwords)
            
            st.write(f"Original: {original_text}")
            st.write(f"Stemming: {' '.join(stemmed_result)}")
            st.write(f"Lemmatization: {' '.join(lemmatized_result)}")
            st.write("\n")

if __name__ == "__main__":
    main()

Overwriting app_lsi.py


**LOAD DATA**

In [2]:
documents = [
    "Teknologi semakin berkembang dan digunakan dalam banyak bidang.",
    "Perangkat pintar membantu kita menghemat energi di rumah.",
    "Pengolahan data membantu perusahaan memahami pelanggan dengan lebih baik.",
    "Sistem otomatis membuat pekerjaan di pabrik menjadi lebih cepat dan efisien.",
    "Kecerdasan buatan digunakan untuk meningkatkan pelayanan kepada pelanggan.",
    "Teknologi buku besar memberikan keamanan dalam transaksi online.",
    "Jaringan perangkat membantu kita mengelola sumber daya dengan lebih mudah.",
    "Realitas virtual dan realitas tertambah digunakan dalam belajar.",
    "Komputasi awan memudahkan akses data di mana saja.",
    "Sistem keamanan melindungi data dari ancaman yang berbahaya.",
    "Pertanian organik semakin populer di kalangan petani muda.",
    "Teknologi pertanian modern membantu meningkatkan hasil panen.",
    "Pupuk organik dapat meningkatkan kesuburan tanah secara alami.",
    "Tanaman padi memerlukan air yang cukup untuk tumbuh dengan baik.",
    "Penggunaan pestisida harus dilakukan dengan hati-hati untuk menghindari dampak lingkungan.",
    "Program subsidi pemerintah mendukung petani kecil untuk mengembangkan usaha mereka.",
    "Budidaya tanaman hortikultura memberikan peluang ekonomi yang baik.",
    "Diversifikasi tanaman penting untuk mengurangi risiko gagal panen.",
    "Irigasi yang efisien sangat penting dalam pertanian di daerah kering.",
    "Penyuluhan pertanian memberikan pengetahuan dan keterampilan kepada petani.",
    "Pendidikan adalah kunci untuk menciptakan generasi yang cerdas dan kompeten.",
    "Sekolah dasar memiliki peranan penting dalam pengembangan karakter anak.",
    "Teknologi informasi mempengaruhi metode pembelajaran di kelas.",
    "Pendidikan inklusif memberikan kesempatan kepada semua anak untuk belajar.",
    "Kurikulum yang relevan membantu siswa mempersiapkan diri untuk masa depan.",
    "Pelatihan guru berkualitas penting untuk meningkatkan standar pendidikan.",
    "Beasiswa pendidikan mendukung siswa berprestasi dari keluarga kurang mampu.",
    "Pembelajaran jarak jauh menjadi alternatif di masa pandemi.",
    "Pendidikan vokasi memberikan keterampilan praktis kepada siswa.",
    "Perpustakaan sekolah merupakan sumber daya penting untuk mendukung pembelajaran.",
    "Penelitian di bidang kesehatan terus berkembang untuk menemukan pengobatan baru.",
    "Pendidikan kesehatan masyarakat menjadi salah satu kunci mencegah penyakit.",
    "Asupan nutrisi yang seimbang sangat penting untuk pertumbuhan anak.",
    "Pandemi global memicu reformasi dalam sistem pelayanan kesehatan.",
    "Pemerataan fasilitas kesehatan di seluruh wilayah menjadi tantangan tersendiri.",
    "Peran dokter dalam kesehatan masyarakat tidak hanya sebagai penyembuh, tetapi juga edukator.",
    "Masalah gizi buruk masih menjadi tantangan besar di beberapa daerah.",
    "Penyakit tidak menular seperti diabetes semakin meningkat di masyarakat urban.",
    "Teknologi medis canggih membantu dalam diagnosis dan perawatan penyakit kompleks.",
    "Program vaksinasi sangat penting untuk mencegah wabah di masyarakat.",
    "Pemerintahan yang transparan menjadi kunci dalam meningkatkan kepercayaan publik.",
    "Desentralisasi membantu daerah untuk lebih mandiri dalam mengelola sumber daya.",
    "Program e-government meningkatkan efisiensi layanan publik.",
    "Perubahan undang-undang dapat berdampak pada tata kelola pemerintahan.",
    "Anggaran negara harus dikelola dengan akuntabilitas yang tinggi.",
    "Kerjasama antara pusat dan daerah sangat penting dalam pemerintahan.",
    "Reformasi birokrasi berfokus pada peningkatan kualitas pelayanan.",
    "Sistem pemilu yang jujur adalah dasar dari demokrasi yang sehat.",
    "Peran aktif masyarakat dalam mengawasi pemerintah sangat dibutuhkan.",
    "Kebijakan luar negeri harus selaras dengan kepentingan nasional."
]

**PRE-PROCCESSING TEXT**

In [3]:
# Step 1: Case folding and cleaning (used in all scenarios)
def case_folding(text):
    text = text.lower()
    return re.sub(r'[^\w\s]', '', text)

# Step 2: Tokenization
def tokenize(text):
    return text.split()

# Step 3: Stopwords removal
def remove_stopwords(tokens):
    return [word for word in tokens if word not in stop_words]

# Step 4: Stemming
def stemming(tokens):
    return [stemmer.stem(word) for word in tokens]

# Step 5: Lemmatization using stanza
def lemmatization(tokens):
    text = ' '.join(tokens)  # Gabungkan token menjadi teks untuk diproses
    doc = nlp(text)
    return [word.lemma for sent in doc.sentences for word in sent.words]

In [4]:
# Preprocessing with stemming
def preprocess_with_stemming(text):
    text = case_folding(text)
    tokens = tokenize(text)
    tokens = remove_stopwords(tokens)
    tokens = stemming(tokens)
    return ' '.join(tokens)

In [5]:
# Preprocessing with lemmatization
def preprocess_with_lemmatization(text):
    text = case_folding(text)
    tokens = tokenize(text)
    tokens = remove_stopwords(tokens)
    tokens = lemmatization(tokens)
    return ' '.join(tokens)

In [6]:
# Menerapkan preprocessing pada setiap dokumen
print("Hasil Preprocessing dengan Stemming dan Lemmatization:")
for doc in documents:
    # Proses case folding
    original_text = case_folding(doc)

    # Tokenisasi
    tokens = tokenize(original_text)

    # Stopwords removal
    tokens_no_stopwords = remove_stopwords(tokens)

    # Proses stemming
    stemmed_result = stemming(tokens_no_stopwords)

    # Proses lemmatization
    lemmatized_result = lemmatization(tokens_no_stopwords)

    # Menampilkan hasil
    print(f"Original\t: {original_text}")
    print(f"Stemming\t: {' '.join(stemmed_result)}")
    print(f"Lemmatization\t: {' '.join(lemmatized_result)}")
    print("\n")

Hasil Preprocessing dengan Stemming dan Lemmatization:
Original	: teknologi semakin berkembang dan digunakan dalam banyak bidang
Stemming	: teknologi kembang bidang
Lemmatization	: teknologi kembang bidang


Original	: perangkat pintar membantu kita menghemat energi di rumah
Stemming	: perangkat pintar bantu hemat energi rumah
Lemmatization	: perangkat pintar bantu hemat energi rumah


Original	: pengolahan data membantu perusahaan memahami pelanggan dengan lebih baik
Stemming	: olah data bantu usaha paham langgan
Lemmatization	: olah data bantu usaha paham langgan


Original	: sistem otomatis membuat pekerjaan di pabrik menjadi lebih cepat dan efisien
Stemming	: sistem otomatis kerja pabrik cepat efisien
Lemmatization	: sistem otomatis pekerja pabrik cepat efisien


Original	: kecerdasan buatan digunakan untuk meningkatkan pelayanan kepada pelanggan
Stemming	: cerdas buat tingkat layan langgan
Lemmatization	: cerdas buat tingkat layan langgan


Original	: teknologi buku besar memberik

**PEMBOBOTAN KATA**

In [7]:
# Fungsi untuk membuat TF-IDF dengan stemming
def tfidf_with_stemming(documents):
    # Preprocessing dengan stemming
    preprocessed_documents = [preprocess_with_stemming(doc) for doc in documents]
    # Buat TF-IDF Vectorizer
    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform(preprocessed_documents)
    return tfidf_matrix, vectorizer

In [8]:
# Fungsi untuk membuat TF-IDF dengan lemmatization
def tfidf_with_lemmatization(documents):
    # Preprocessing dengan lemmatization
    preprocessed_documents = [preprocess_with_lemmatization(doc) for doc in documents]
    # Buat TF-IDF Vectorizer
    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform(preprocessed_documents)
    return tfidf_matrix, vectorizer

**QUERY PENCARIAN DOKUMEN**

In [9]:
query = "Asupan nutrisi apa saja yang dibutuhkan untuk pertumbuhan anak?"

**Stemming**

In [10]:
# Fungsi untuk LSI dengan stemming
def lsi_with_stemming(documents, query, random_state=42):
    # Buat TF-IDF dengan stemming
    tfidf_matrix, vectorizer = tfidf_with_stemming(documents)

    # Lakukan Truncated SVD (LSI)
    svd = TruncatedSVD(n_components=2, random_state=random_state)
    lsi_matrix = svd.fit_transform(tfidf_matrix)

    # Preprocessing query dengan stemming
    preprocessed_query = preprocess_with_stemming(query)
    query_vec = vectorizer.transform([preprocessed_query])
    query_lsi = svd.transform(query_vec)

    # Hitung kesamaan cosine antara query dan dokumen
    similarities = cosine_similarity(query_lsi, lsi_matrix)

    # Urutkan dokumen berdasarkan kesamaan
    ranked_docs = np.argsort(similarities[0])[::-1]

    # Ambil top-10 dokumen
    top_10_docs = ranked_docs[:10]

    # Tampilkan hasil
    print("Pencarian dengan LSI (Stemming):")
    print("Query:", query)
    print("\nTop Documents:")
    for i, idx in enumerate(top_10_docs):
        print(f"{i + 1}. Doc {idx+1} \t (Similarity: {similarities[0][idx]:.4f}) \t: {documents[idx]}")

# Lakukan pencarian dengan stemming
lsi_with_stemming(documents, query)

Pencarian dengan LSI (Stemming):
Query: Asupan nutrisi apa saja yang dibutuhkan untuk pertumbuhan anak?

Top Documents:
1. Doc 29 	 (Similarity: 0.9999) 	: Pendidikan vokasi memberikan keterampilan praktis kepada siswa.
2. Doc 24 	 (Similarity: 0.9998) 	: Pendidikan inklusif memberikan kesempatan kepada semua anak untuk belajar.
3. Doc 22 	 (Similarity: 0.9902) 	: Sekolah dasar memiliki peranan penting dalam pengembangan karakter anak.
4. Doc 27 	 (Similarity: 0.9813) 	: Beasiswa pendidikan mendukung siswa berprestasi dari keluarga kurang mampu.
5. Doc 28 	 (Similarity: 0.9741) 	: Pembelajaran jarak jauh menjadi alternatif di masa pandemi.
6. Doc 1 	 (Similarity: 0.9683) 	: Teknologi semakin berkembang dan digunakan dalam banyak bidang.
7. Doc 31 	 (Similarity: 0.9683) 	: Penelitian di bidang kesehatan terus berkembang untuk menemukan pengobatan baru.
8. Doc 10 	 (Similarity: 0.9461) 	: Sistem keamanan melindungi data dari ancaman yang berbahaya.
9. Doc 33 	 (Similarity: 0.9401) 	: Asu

**Lemmatization**

In [11]:
# Fungsi untuk LSI dengan lemmatization
def lsi_with_lemmatization(documents, query, random_state=42):
    # Buat TF-IDF dengan lemmatization
    tfidf_matrix, vectorizer = tfidf_with_lemmatization(documents)

    # Lakukan Truncated SVD (LSI)
    svd = TruncatedSVD(n_components=2, random_state=random_state)
    lsi_matrix = svd.fit_transform(tfidf_matrix)

    # Preprocessing query dengan lemmatization
    preprocessed_query = preprocess_with_lemmatization(query)
    query_vec = vectorizer.transform([preprocessed_query])
    query_lsi = svd.transform(query_vec)

    # Hitung kesamaan cosine antara query dan dokumen
    similarities = cosine_similarity(query_lsi, lsi_matrix)

    # Urutkan dokumen berdasarkan kesamaan
    ranked_docs = np.argsort(similarities[0])[::-1]

    # Ambil top-10 dokumen
    top_10_docs = ranked_docs[:10]

    # Tampilkan hasil
    print("Pencarian dengan LSI (Lemmatization):")
    print("Query:", query)
    print("\nTop Documents:")
    for i, idx in enumerate(top_10_docs):
        print(f"{i + 1}. Doc {idx+1} \t (Similarity: {similarities[0][idx]:.4f}) \t: {documents[idx]}")

# Lakukan pencarian dengan lemmatization
lsi_with_lemmatization(documents, query)

Pencarian dengan LSI (Lemmatization):
Query: Asupan nutrisi apa saja yang dibutuhkan untuk pertumbuhan anak?

Top Documents:
1. Doc 24 	 (Similarity: 0.9987) 	: Pendidikan inklusif memberikan kesempatan kepada semua anak untuk belajar.
2. Doc 16 	 (Similarity: 0.9984) 	: Program subsidi pemerintah mendukung petani kecil untuk mengembangkan usaha mereka.
3. Doc 22 	 (Similarity: 0.9983) 	: Sekolah dasar memiliki peranan penting dalam pengembangan karakter anak.
4. Doc 1 	 (Similarity: 0.9976) 	: Teknologi semakin berkembang dan digunakan dalam banyak bidang.
5. Doc 29 	 (Similarity: 0.9947) 	: Pendidikan vokasi memberikan keterampilan praktis kepada siswa.
6. Doc 27 	 (Similarity: 0.9910) 	: Beasiswa pendidikan mendukung siswa berprestasi dari keluarga kurang mampu.
7. Doc 4 	 (Similarity: 0.9669) 	: Sistem otomatis membuat pekerjaan di pabrik menjadi lebih cepat dan efisien.
8. Doc 28 	 (Similarity: 0.9606) 	: Pembelajaran jarak jauh menjadi alternatif di masa pandemi.
9. Doc 10 	 (Sim

**Kesimpulan**


> Pra-pemrosesan dokumen menggunakan lemmatization menghasilkan nilai similarity yang lebih tinggi dibandingkan menggunakan stemming. Oleh karena itu lemmatization akan digunakan untuk skenario selanjutnya.








# SKENARIO 2: MENGUBAH JUMLAH TOPIK LATEN (N_COMPONENTS)

> Mesin pencarian belum menghasilkan pencarian yang akurat berdasarkan query yang diberikan, oleh karena itu dilakukan skenario untuk mengubah jumlah topik laten [2, 5, 10, 20].


> Hipotesis: Menggunakan lebih banyak topik laten akan menghasilkan pencarian yang lebih akurat, tetapi setelah titik tertentu, penambahan topik laten mungkin tidak lagi memberikan peningkatan yang signifikan.

**n_components=2**

In [12]:
# Fungsi untuk LSI dengan lemmatization
def lsi_with_lemmatization(documents, query, random_state=42):
    # Buat TF-IDF dengan lemmatization
    tfidf_matrix, vectorizer = tfidf_with_lemmatization(documents)

    # Lakukan Truncated SVD (LSI)
    svd = TruncatedSVD(n_components=2, random_state=random_state)
    lsi_matrix = svd.fit_transform(tfidf_matrix)

    # Preprocessing query dengan lemmatization
    preprocessed_query = preprocess_with_lemmatization(query)
    query_vec = vectorizer.transform([preprocessed_query])
    query_lsi = svd.transform(query_vec)

    # Hitung kesamaan cosine antara query dan dokumen
    similarities = cosine_similarity(query_lsi, lsi_matrix)

    # Urutkan dokumen berdasarkan kesamaan
    ranked_docs = np.argsort(similarities[0])[::-1]

    # Ambil top-10 dokumen
    top_10_docs = ranked_docs[:10]

    # Tampilkan hasil
    print("Pencarian dengan LSI (n_components=2):")
    print("Query:", query)
    print("\nTop Documents:")
    for i, idx in enumerate(top_10_docs):
        print(f"{i + 1}. Doc {idx+1} \t (Similarity: {similarities[0][idx]:.4f}) \t: {documents[idx]}")

# Lakukan pencarian dengan lemmatization
lsi_with_lemmatization(documents, query)


Pencarian dengan LSI (n_components=2):
Query: Asupan nutrisi apa saja yang dibutuhkan untuk pertumbuhan anak?

Top Documents:
1. Doc 24 	 (Similarity: 0.9987) 	: Pendidikan inklusif memberikan kesempatan kepada semua anak untuk belajar.
2. Doc 16 	 (Similarity: 0.9984) 	: Program subsidi pemerintah mendukung petani kecil untuk mengembangkan usaha mereka.
3. Doc 22 	 (Similarity: 0.9983) 	: Sekolah dasar memiliki peranan penting dalam pengembangan karakter anak.
4. Doc 1 	 (Similarity: 0.9976) 	: Teknologi semakin berkembang dan digunakan dalam banyak bidang.
5. Doc 29 	 (Similarity: 0.9947) 	: Pendidikan vokasi memberikan keterampilan praktis kepada siswa.
6. Doc 27 	 (Similarity: 0.9910) 	: Beasiswa pendidikan mendukung siswa berprestasi dari keluarga kurang mampu.
7. Doc 4 	 (Similarity: 0.9669) 	: Sistem otomatis membuat pekerjaan di pabrik menjadi lebih cepat dan efisien.
8. Doc 28 	 (Similarity: 0.9606) 	: Pembelajaran jarak jauh menjadi alternatif di masa pandemi.
9. Doc 10 	 (Si

**n_components=5**

In [13]:
# Fungsi untuk LSI dengan lemmatization
def lsi_with_lemmatization(documents, query, random_state=42):
    # Buat TF-IDF dengan lemmatization
    tfidf_matrix, vectorizer = tfidf_with_lemmatization(documents)

    # Lakukan Truncated SVD (LSI)
    svd = TruncatedSVD(n_components=5, random_state=random_state)
    lsi_matrix = svd.fit_transform(tfidf_matrix)

    # Preprocessing query dengan lemmatization
    preprocessed_query = preprocess_with_lemmatization(query)
    query_vec = vectorizer.transform([preprocessed_query])
    query_lsi = svd.transform(query_vec)

    # Hitung kesamaan cosine antara query dan dokumen
    similarities = cosine_similarity(query_lsi, lsi_matrix)

    # Urutkan dokumen berdasarkan kesamaan
    ranked_docs = np.argsort(similarities[0])[::-1]

    # Ambil top-10 dokumen
    top_10_docs = ranked_docs[:10]

    # Tampilkan hasil
    print("Pencarian dengan LSI (n_components=5):")
    print("Query:", query)
    print("\nTop Documents:")
    for i, idx in enumerate(top_10_docs):
        print(f"{i + 1}. Doc {idx+1} \t (Similarity: {similarities[0][idx]:.4f}) \t: {documents[idx]}")

# Lakukan pencarian dengan lemmatization
lsi_with_lemmatization(documents, query)


Pencarian dengan LSI (n_components=5):
Query: Asupan nutrisi apa saja yang dibutuhkan untuk pertumbuhan anak?

Top Documents:
1. Doc 33 	 (Similarity: 0.9387) 	: Asupan nutrisi yang seimbang sangat penting untuk pertumbuhan anak.
2. Doc 22 	 (Similarity: 0.8562) 	: Sekolah dasar memiliki peranan penting dalam pengembangan karakter anak.
3. Doc 24 	 (Similarity: 0.8147) 	: Pendidikan inklusif memberikan kesempatan kepada semua anak untuk belajar.
4. Doc 28 	 (Similarity: 0.7833) 	: Pembelajaran jarak jauh menjadi alternatif di masa pandemi.
5. Doc 8 	 (Similarity: 0.7737) 	: Realitas virtual dan realitas tertambah digunakan dalam belajar.
6. Doc 14 	 (Similarity: 0.6496) 	: Tanaman padi memerlukan air yang cukup untuk tumbuh dengan baik.
7. Doc 32 	 (Similarity: 0.6169) 	: Pendidikan kesehatan masyarakat menjadi salah satu kunci mencegah penyakit.
8. Doc 27 	 (Similarity: 0.6072) 	: Beasiswa pendidikan mendukung siswa berprestasi dari keluarga kurang mampu.
9. Doc 31 	 (Similarity: 0.60

**n_components=10**

In [14]:
# Fungsi untuk LSI dengan lemmatization
def lsi_with_lemmatization(documents, query, random_state=42):
    # Buat TF-IDF dengan lemmatization
    tfidf_matrix, vectorizer = tfidf_with_lemmatization(documents)

    # Lakukan Truncated SVD (LSI)
    svd = TruncatedSVD(n_components=10, random_state=random_state)
    lsi_matrix = svd.fit_transform(tfidf_matrix)

    # Preprocessing query dengan lemmatization
    preprocessed_query = preprocess_with_lemmatization(query)
    query_vec = vectorizer.transform([preprocessed_query])
    query_lsi = svd.transform(query_vec)

    # Hitung kesamaan cosine antara query dan dokumen
    similarities = cosine_similarity(query_lsi, lsi_matrix)

    # Urutkan dokumen berdasarkan kesamaan
    ranked_docs = np.argsort(similarities[0])[::-1]

    # Ambil top-10 dokumen
    top_10_docs = ranked_docs[:10]

    # Tampilkan hasil
    print("Pencarian dengan LSI (n_components=10):")
    print("Query:", query)
    print("\nTop Documents:")
    for i, idx in enumerate(top_10_docs):
        print(f"{i + 1}. Doc {idx+1} \t (Similarity: {similarities[0][idx]:.4f}) \t: {documents[idx]}")

# Lakukan pencarian dengan lemmatization
lsi_with_lemmatization(documents, query)


Pencarian dengan LSI (n_components=10):
Query: Asupan nutrisi apa saja yang dibutuhkan untuk pertumbuhan anak?

Top Documents:
1. Doc 33 	 (Similarity: 0.9893) 	: Asupan nutrisi yang seimbang sangat penting untuk pertumbuhan anak.
2. Doc 14 	 (Similarity: 0.9148) 	: Tanaman padi memerlukan air yang cukup untuk tumbuh dengan baik.
3. Doc 17 	 (Similarity: 0.8919) 	: Budidaya tanaman hortikultura memberikan peluang ekonomi yang baik.
4. Doc 18 	 (Similarity: 0.7168) 	: Diversifikasi tanaman penting untuk mengurangi risiko gagal panen.
5. Doc 22 	 (Similarity: 0.6314) 	: Sekolah dasar memiliki peranan penting dalam pengembangan karakter anak.
6. Doc 50 	 (Similarity: 0.5214) 	: Kebijakan luar negeri harus selaras dengan kepentingan nasional.
7. Doc 24 	 (Similarity: 0.4424) 	: Pendidikan inklusif memberikan kesempatan kepada semua anak untuk belajar.
8. Doc 49 	 (Similarity: 0.3240) 	: Peran aktif masyarakat dalam mengawasi pemerintah sangat dibutuhkan.
9. Doc 37 	 (Similarity: 0.2706) 	:

**n_components=20**

In [15]:
# Fungsi untuk LSI dengan lemmatization
def lsi_with_lemmatization(documents, query, random_state=42):
    # Buat TF-IDF dengan lemmatization
    tfidf_matrix, vectorizer = tfidf_with_lemmatization(documents)

    # Lakukan Truncated SVD (LSI)
    svd = TruncatedSVD(n_components=20, random_state=random_state)
    lsi_matrix = svd.fit_transform(tfidf_matrix)

    # Preprocessing query dengan lemmatization
    preprocessed_query = preprocess_with_lemmatization(query)
    query_vec = vectorizer.transform([preprocessed_query])
    query_lsi = svd.transform(query_vec)

    # Hitung kesamaan cosine antara query dan dokumen
    similarities = cosine_similarity(query_lsi, lsi_matrix)

    # Urutkan dokumen berdasarkan kesamaan
    ranked_docs = np.argsort(similarities[0])[::-1]

    # Ambil top-10 dokumen
    top_10_docs = ranked_docs[:10]

    # Tampilkan hasil
    print("Pencarian dengan LSI (n_components=20):")
    print("Query:", query)
    print("\nTop Documents:")
    for i, idx in enumerate(top_10_docs):
        print(f"{i + 1}. Doc {idx+1} \t (Similarity: {similarities[0][idx]:.4f}) \t: {documents[idx]}")

# Lakukan pencarian dengan lemmatization
lsi_with_lemmatization(documents, query)


Pencarian dengan LSI (n_components=20):
Query: Asupan nutrisi apa saja yang dibutuhkan untuk pertumbuhan anak?

Top Documents:
1. Doc 33 	 (Similarity: 0.9815) 	: Asupan nutrisi yang seimbang sangat penting untuk pertumbuhan anak.
2. Doc 14 	 (Similarity: 0.6094) 	: Tanaman padi memerlukan air yang cukup untuk tumbuh dengan baik.
3. Doc 24 	 (Similarity: 0.5476) 	: Pendidikan inklusif memberikan kesempatan kepada semua anak untuk belajar.
4. Doc 22 	 (Similarity: 0.5101) 	: Sekolah dasar memiliki peranan penting dalam pengembangan karakter anak.
5. Doc 49 	 (Similarity: 0.1544) 	: Peran aktif masyarakat dalam mengawasi pemerintah sangat dibutuhkan.
6. Doc 2 	 (Similarity: 0.0917) 	: Perangkat pintar membantu kita menghemat energi di rumah.
7. Doc 36 	 (Similarity: 0.0884) 	: Peran dokter dalam kesehatan masyarakat tidak hanya sebagai penyembuh, tetapi juga edukator.
8. Doc 45 	 (Similarity: 0.0692) 	: Anggaran negara harus dikelola dengan akuntabilitas yang tinggi.
9. Doc 46 	 (Similar

**Kesimpulan**


> Mesin pencarian bekerja dengan baik saat nilai n_components=10 (analisis lebih lanjut)



# SKENARIO 3: VARIASI PEMBOBOTAN

**TF-IDF**

In [16]:
# Fungsi untuk LSI dengan lemmatization
def lsi_with_lemmatization(documents, query, random_state=42):
    # Buat TF-IDF dengan lemmatization
    tfidf_matrix, vectorizer = tfidf_with_lemmatization(documents)

    # Lakukan Truncated SVD (LSI)
    svd = TruncatedSVD(n_components=20, random_state=random_state)
    lsi_matrix = svd.fit_transform(tfidf_matrix)

    # Preprocessing query dengan lemmatization
    preprocessed_query = preprocess_with_lemmatization(query)
    query_vec = vectorizer.transform([preprocessed_query])
    query_lsi = svd.transform(query_vec)

    # Hitung kesamaan cosine antara query dan dokumen
    similarities = cosine_similarity(query_lsi, lsi_matrix)

    # Urutkan dokumen berdasarkan kesamaan
    ranked_docs = np.argsort(similarities[0])[::-1]

    # Ambil top-10 dokumen
    top_10_docs = ranked_docs[:10]

    # Tampilkan hasil
    print("Pencarian dengan LSI (TF-IDF):")
    print("Query:", query)
    print("\nTop Documents:")
    for i, idx in enumerate(top_10_docs):
        print(f"{i + 1}. Doc {idx+1} \t (Similarity: {similarities[0][idx]:.4f}) \t: {documents[idx]}")

# Lakukan pencarian dengan lemmatization
lsi_with_lemmatization(documents, query)


Pencarian dengan LSI (TF-IDF):
Query: Asupan nutrisi apa saja yang dibutuhkan untuk pertumbuhan anak?

Top Documents:
1. Doc 33 	 (Similarity: 0.9815) 	: Asupan nutrisi yang seimbang sangat penting untuk pertumbuhan anak.
2. Doc 14 	 (Similarity: 0.6094) 	: Tanaman padi memerlukan air yang cukup untuk tumbuh dengan baik.
3. Doc 24 	 (Similarity: 0.5476) 	: Pendidikan inklusif memberikan kesempatan kepada semua anak untuk belajar.
4. Doc 22 	 (Similarity: 0.5101) 	: Sekolah dasar memiliki peranan penting dalam pengembangan karakter anak.
5. Doc 49 	 (Similarity: 0.1544) 	: Peran aktif masyarakat dalam mengawasi pemerintah sangat dibutuhkan.
6. Doc 2 	 (Similarity: 0.0917) 	: Perangkat pintar membantu kita menghemat energi di rumah.
7. Doc 36 	 (Similarity: 0.0884) 	: Peran dokter dalam kesehatan masyarakat tidak hanya sebagai penyembuh, tetapi juga edukator.
8. Doc 45 	 (Similarity: 0.0692) 	: Anggaran negara harus dikelola dengan akuntabilitas yang tinggi.
9. Doc 46 	 (Similarity: 0.06

**Term Frequency (TF) Biasa**

In [17]:
# Fungsi untuk menghitung TF biasa
def tf_with_lemmatization(documents):
    # Preprocessing dengan lemmatization
    preprocessed_documents = [preprocess_with_lemmatization(doc) for doc in documents]
    # Buat CountVectorizer untuk menghitung TF biasa
    vectorizer = CountVectorizer()
    tf_matrix = vectorizer.fit_transform(preprocessed_documents)
    return tf_matrix, vectorizer

# Fungsi LSI dengan TF Biasa
def lsi_with_tf_lemmatization(documents, query, random_state=42):
    # Buat TF biasa dengan lemmatization
    tf_matrix, vectorizer = tf_with_lemmatization(documents)

    # Lakukan Truncated SVD (LSI)
    svd = TruncatedSVD(n_components=20, random_state=random_state)
    lsi_matrix = svd.fit_transform(tf_matrix)

    # Preprocessing query dengan lemmatization
    preprocessed_query = preprocess_with_lemmatization(query)
    query_vec = vectorizer.transform([preprocessed_query])
    query_lsi = svd.transform(query_vec)

    # Hitung kesamaan cosine antara query dan dokumen
    similarities = cosine_similarity(query_lsi, lsi_matrix)

    # Urutkan dokumen berdasarkan kesamaan
    ranked_docs = np.argsort(similarities[0])[::-1]

    # Ambil top-10 dokumen
    top_10_docs = ranked_docs[:10]

    # Tampilkan hasil
    print("Pencarian dengan LSI (TF Biasa):")
    print("Query:", query)
    print("\nTop Documents:")
    for i, idx in enumerate(top_10_docs):
        print(f"{i + 1}. Doc {idx+1} \t (Similarity: {similarities[0][idx]:.4f}) \t: {documents[idx]}")

# Lakukan pencarian
lsi_with_tf_lemmatization(documents, query)

Pencarian dengan LSI (TF Biasa):
Query: Asupan nutrisi apa saja yang dibutuhkan untuk pertumbuhan anak?

Top Documents:
1. Doc 33 	 (Similarity: 0.9833) 	: Asupan nutrisi yang seimbang sangat penting untuk pertumbuhan anak.
2. Doc 22 	 (Similarity: 0.5951) 	: Sekolah dasar memiliki peranan penting dalam pengembangan karakter anak.
3. Doc 14 	 (Similarity: 0.5935) 	: Tanaman padi memerlukan air yang cukup untuk tumbuh dengan baik.
4. Doc 24 	 (Similarity: 0.5472) 	: Pendidikan inklusif memberikan kesempatan kepada semua anak untuk belajar.
5. Doc 50 	 (Similarity: 0.4201) 	: Kebijakan luar negeri harus selaras dengan kepentingan nasional.
6. Doc 17 	 (Similarity: 0.2642) 	: Budidaya tanaman hortikultura memberikan peluang ekonomi yang baik.
7. Doc 49 	 (Similarity: 0.2029) 	: Peran aktif masyarakat dalam mengawasi pemerintah sangat dibutuhkan.
8. Doc 39 	 (Similarity: 0.1394) 	: Teknologi medis canggih membantu dalam diagnosis dan perawatan penyakit kompleks.
9. Doc 28 	 (Similarity: 0.

**Binary Term Frequency**

In [18]:
# Fungsi untuk menghitung Binary TF
def binary_tf_with_lemmatization(documents):
    # Preprocessing dengan lemmatization
    preprocessed_documents = [preprocess_with_lemmatization(doc) for doc in documents]
    # Buat CountVectorizer dengan binary=True untuk binary TF
    vectorizer = CountVectorizer(binary=True)
    binary_tf_matrix = vectorizer.fit_transform(preprocessed_documents)
    return binary_tf_matrix, vectorizer

# Fungsi LSI dengan Binary TF
def lsi_with_binary_tf_lemmatization(documents, query, random_state=42):
    # Buat binary TF dengan lemmatization
    tf_matrix, vectorizer = binary_tf_with_lemmatization(documents)

    # Lakukan Truncated SVD (LSI)
    svd = TruncatedSVD(n_components=20, random_state=random_state)
    lsi_matrix = svd.fit_transform(tf_matrix)

    # Preprocessing query dengan lemmatization
    preprocessed_query = preprocess_with_lemmatization(query)
    query_vec = vectorizer.transform([preprocessed_query])
    query_lsi = svd.transform(query_vec)

    # Hitung kesamaan cosine antara query dan dokumen
    similarities = cosine_similarity(query_lsi, lsi_matrix)

    # Urutkan dokumen berdasarkan kesamaan
    ranked_docs = np.argsort(similarities[0])[::-1]

    # Ambil top-10 dokumen
    top_10_docs = ranked_docs[:10]

    # Tampilkan hasil
    print("Pencarian dengan LSI (Binary TF):")
    print("Query:", query)
    print("\nTop Documents:")
    for i, idx in enumerate(top_10_docs):
        print(f"{i + 1}. Doc {idx+1} \t (Similarity: {similarities[0][idx]:.4f}) \t: {documents[idx]}")

# Lakukan pencarian
lsi_with_binary_tf_lemmatization(documents, query)

Pencarian dengan LSI (Binary TF):
Query: Asupan nutrisi apa saja yang dibutuhkan untuk pertumbuhan anak?

Top Documents:
1. Doc 33 	 (Similarity: 0.9827) 	: Asupan nutrisi yang seimbang sangat penting untuk pertumbuhan anak.
2. Doc 14 	 (Similarity: 0.5926) 	: Tanaman padi memerlukan air yang cukup untuk tumbuh dengan baik.
3. Doc 22 	 (Similarity: 0.5882) 	: Sekolah dasar memiliki peranan penting dalam pengembangan karakter anak.
4. Doc 24 	 (Similarity: 0.5422) 	: Pendidikan inklusif memberikan kesempatan kepada semua anak untuk belajar.
5. Doc 50 	 (Similarity: 0.4290) 	: Kebijakan luar negeri harus selaras dengan kepentingan nasional.
6. Doc 17 	 (Similarity: 0.2649) 	: Budidaya tanaman hortikultura memberikan peluang ekonomi yang baik.
7. Doc 49 	 (Similarity: 0.1928) 	: Peran aktif masyarakat dalam mengawasi pemerintah sangat dibutuhkan.
8. Doc 39 	 (Similarity: 0.1605) 	: Teknologi medis canggih membantu dalam diagnosis dan perawatan penyakit kompleks.
9. Doc 28 	 (Similarity: 0

**Logarithmic Normalization Term Frequency**

In [19]:
# Fungsi untuk menghitung Logarithmic TF
def log_tf_with_lemmatization(documents):
    # Preprocessing dengan lemmatization
    preprocessed_documents = [preprocess_with_lemmatization(doc) for doc in documents]
    # Buat CountVectorizer untuk menghitung log TF
    vectorizer = CountVectorizer()
    tf_matrix = vectorizer.fit_transform(preprocessed_documents).toarray()
    # Terapkan log untuk setiap elemen dari TF matrix
    log_tf_matrix = np.log1p(tf_matrix)  # log(1 + tf)
    return log_tf_matrix, vectorizer

# Fungsi LSI dengan log TF
def lsi_with_log_tf_lemmatization(documents, query, random_state=42):
    # Buat TF biasa dengan lemmatization
    tf_matrix, vectorizer = log_tf_with_lemmatization(documents)

    # Lakukan Truncated SVD (LSI)
    svd = TruncatedSVD(n_components=20, random_state=random_state)
    lsi_matrix = svd.fit_transform(tf_matrix)

    # Preprocessing query dengan lemmatization
    preprocessed_query = preprocess_with_lemmatization(query)
    query_vec = vectorizer.transform([preprocessed_query])
    query_lsi = svd.transform(query_vec)

    # Hitung kesamaan cosine antara query dan dokumen
    similarities = cosine_similarity(query_lsi, lsi_matrix)

    # Urutkan dokumen berdasarkan kesamaan
    ranked_docs = np.argsort(similarities[0])[::-1]

    # Ambil top-10 dokumen
    top_10_docs = ranked_docs[:10]

    # Tampilkan hasil
    print("Pencarian dengan LSI (Log TF):")
    print("Query:", query)
    print("\nTop Documents:")
    for i, idx in enumerate(top_10_docs):
        print(f"{i + 1}. Doc {idx+1} \t (Similarity: {similarities[0][idx]:.4f}) \t: {documents[idx]}")

# Lakukan pencarian
lsi_with_log_tf_lemmatization(documents, query)

Pencarian dengan LSI (Log TF):
Query: Asupan nutrisi apa saja yang dibutuhkan untuk pertumbuhan anak?

Top Documents:
1. Doc 33 	 (Similarity: 0.9829) 	: Asupan nutrisi yang seimbang sangat penting untuk pertumbuhan anak.
2. Doc 22 	 (Similarity: 0.5987) 	: Sekolah dasar memiliki peranan penting dalam pengembangan karakter anak.
3. Doc 14 	 (Similarity: 0.5938) 	: Tanaman padi memerlukan air yang cukup untuk tumbuh dengan baik.
4. Doc 24 	 (Similarity: 0.5303) 	: Pendidikan inklusif memberikan kesempatan kepada semua anak untuk belajar.
5. Doc 50 	 (Similarity: 0.3773) 	: Kebijakan luar negeri harus selaras dengan kepentingan nasional.
6. Doc 17 	 (Similarity: 0.2686) 	: Budidaya tanaman hortikultura memberikan peluang ekonomi yang baik.
7. Doc 49 	 (Similarity: 0.1971) 	: Peran aktif masyarakat dalam mengawasi pemerintah sangat dibutuhkan.
8. Doc 39 	 (Similarity: 0.1573) 	: Teknologi medis canggih membantu dalam diagnosis dan perawatan penyakit kompleks.
9. Doc 5 	 (Similarity: 0.106

**Double Normalization 0.5 Term Frequency**

In [20]:
# Fungsi untuk menghitung Augmented TF
def augmented_tf_with_lemmatization(documents):
    # Preprocessing dengan lemmatization
    preprocessed_documents = [preprocess_with_lemmatization(doc) for doc in documents]
    # Buat CountVectorizer untuk menghitung augmented TF
    vectorizer = CountVectorizer()
    tf_matrix = vectorizer.fit_transform(preprocessed_documents).toarray()
    # Terapkan augmented TF untuk setiap dokumen
    augmented_tf_matrix = 0.5 + 0.5 * (tf_matrix / tf_matrix.max(axis=1, keepdims=True))
    return augmented_tf_matrix, vectorizer

# Fungsi LSI dengan augmented TF
def lsi_with_augmented_tf_lemmatization(documents, query, random_state=42):
    # Buat augmented TF dengan lemmatization
    tf_matrix, vectorizer = augmented_tf_with_lemmatization(documents)

    # Lakukan Truncated SVD (LSI)
    svd = TruncatedSVD(n_components=20, random_state=random_state)
    lsi_matrix = svd.fit_transform(tf_matrix)

    # Preprocessing query dengan lemmatization
    preprocessed_query = preprocess_with_lemmatization(query)
    query_vec = vectorizer.transform([preprocessed_query])
    query_lsi = svd.transform(query_vec)

    # Hitung kesamaan cosine antara query dan dokumen
    similarities = cosine_similarity(query_lsi, lsi_matrix)

    # Urutkan dokumen berdasarkan kesamaan
    ranked_docs = np.argsort(similarities[0])[::-1]

    # Ambil top-10 dokumen
    top_10_docs = ranked_docs[:10]

    # Tampilkan hasil
    print("Pencarian dengan LSI (augmented TF):")
    print("Query:", query)
    print("\nTop Documents:")
    for i, idx in enumerate(top_10_docs):
        print(f"{i + 1}. Doc {idx+1} \t (Similarity: {similarities[0][idx]:.4f}) \t: {documents[idx]}")

# Lakukan pencarian
lsi_with_augmented_tf_lemmatization(documents, query)

Pencarian dengan LSI (augmented TF):
Query: Asupan nutrisi apa saja yang dibutuhkan untuk pertumbuhan anak?

Top Documents:
1. Doc 33 	 (Similarity: 0.3813) 	: Asupan nutrisi yang seimbang sangat penting untuk pertumbuhan anak.
2. Doc 22 	 (Similarity: 0.3596) 	: Sekolah dasar memiliki peranan penting dalam pengembangan karakter anak.
3. Doc 24 	 (Similarity: 0.3258) 	: Pendidikan inklusif memberikan kesempatan kepada semua anak untuk belajar.
4. Doc 14 	 (Similarity: 0.3215) 	: Tanaman padi memerlukan air yang cukup untuk tumbuh dengan baik.
5. Doc 17 	 (Similarity: 0.2956) 	: Budidaya tanaman hortikultura memberikan peluang ekonomi yang baik.
6. Doc 49 	 (Similarity: 0.2920) 	: Peran aktif masyarakat dalam mengawasi pemerintah sangat dibutuhkan.
7. Doc 50 	 (Similarity: 0.2914) 	: Kebijakan luar negeri harus selaras dengan kepentingan nasional.
8. Doc 39 	 (Similarity: 0.2891) 	: Teknologi medis canggih membantu dalam diagnosis dan perawatan penyakit kompleks.
9. Doc 28 	 (Similarity

# SKENARIO 4: EUCLIDEAN DISTANCE


> Hipotesis: Cosine Similarity lebih baik untuk menghitung kemiripan teks karena lebih sensitif terhadap arah vektor daripada panjang vektor, yang bisa lebih relevan untuk teks.



In [21]:
# Fungsi untuk LSI dengan Lemmatization dan Euclidean Distance
def lsi_with_lemmatization_euclidean(documents, query, random_state=42):
    # Buat TF-IDF dengan lemmatization
    tfidf_matrix, vectorizer = tf_with_lemmatization(documents)

    # Lakukan Truncated SVD (LSI)
    svd = TruncatedSVD(n_components=10, random_state=random_state)  # Gunakan lebih banyak komponen jika dibutuhkan
    lsi_matrix = svd.fit_transform(tfidf_matrix)

    # Preprocessing query dengan lemmatization
    preprocessed_query = preprocess_with_lemmatization(query)
    query_vec = vectorizer.transform([preprocessed_query])
    query_lsi = svd.transform(query_vec)

    # Hitung Euclidean Distance antara query dan dokumen
    distances = [euclidean(query_lsi[0], doc_vec) for doc_vec in lsi_matrix]

    # Urutkan dokumen berdasarkan jarak terdekat (semakin kecil semakin mirip)
    ranked_docs = np.argsort(distances)

    # Ambil top-10 dokumen
    top_10_docs = ranked_docs[:10]

    # Tampilkan hasil
    print("Pencarian dengan LSI (Lemmatization, Euclidean Distance):")
    print("Query:", query)
    print("\nTop Documents:")
    for i, idx in enumerate(top_10_docs):
      print(f"{i + 1}. Doc {idx+1} \t (Euclidean Distance: {distances[idx]:.4f}) \t: {documents[idx]}")

# Lakukan pencarian dengan lemmatization menggunakan Euclidean Distance
lsi_with_lemmatization_euclidean(documents, query)

Pencarian dengan LSI (Lemmatization, Euclidean Distance):
Query: Asupan nutrisi apa saja yang dibutuhkan untuk pertumbuhan anak?

Top Documents:
1. Doc 33 	 (Euclidean Distance: 0.1870) 	: Asupan nutrisi yang seimbang sangat penting untuk pertumbuhan anak.
2. Doc 14 	 (Euclidean Distance: 0.4041) 	: Tanaman padi memerlukan air yang cukup untuk tumbuh dengan baik.
3. Doc 17 	 (Euclidean Distance: 0.4695) 	: Budidaya tanaman hortikultura memberikan peluang ekonomi yang baik.
4. Doc 50 	 (Euclidean Distance: 0.5280) 	: Kebijakan luar negeri harus selaras dengan kepentingan nasional.
5. Doc 18 	 (Euclidean Distance: 0.6450) 	: Diversifikasi tanaman penting untuk mengurangi risiko gagal panen.
6. Doc 1 	 (Euclidean Distance: 0.8348) 	: Teknologi semakin berkembang dan digunakan dalam banyak bidang.
7. Doc 15 	 (Euclidean Distance: 0.8545) 	: Penggunaan pestisida harus dilakukan dengan hati-hati untuk menghindari dampak lingkungan.
8. Doc 45 	 (Euclidean Distance: 0.8920) 	: Anggaran negara 